In [ ]:
!pip -q install ultralytics

In [ ]:
import os
from ultralytics import YOLO
import cv2

In [ ]:
# Load a model
model = YOLO("yolov8n.pt")

# Define augmentation parameters
augmentation_params = {
    'hsv_h': 0.015,  # HSV-Hue augmentation (fraction)
    'hsv_s': 0.7,    # HSV-Saturation augmentation (fraction)
    'hsv_v': 0.4,    # HSV-Value augmentation (fraction)
    'degrees': 0.0,  # Rotation (degrees)
    'translate': 0.2,  # Translation (fraction)
    'scale': 0.5,    # Scale (fraction)
    'shear': 0.0,    # Shear (degrees)
    'perspective': 0.0,  # Perspective (fraction)
    'flipud': 0.5,   # Flip up-down (probability)
    'fliplr': 0.5,   # Flip left-right (probability)
    'mosaic': 1.0,   # Mosaic augmentation (probability)
    'mixup': 0.0,    # Mixup augmentation (probability)
    'copy_paste': 0.0,  # Copy-Paste augmentation (probability)
    'crop_fraction': 0.3  # Crop fraction
}

# Use the model with augmentation
result = model.train(
    data="data_config.yaml",
    epochs=200,
    imgsz=840,
    augment=True,  # Enable built-in augmentations
    **augmentation_params  # Apply custom augmentation parameters
)

In [ ]:
# Define paths
input_path = os.path.join('.', 'input')  # Directory for video or image files
output_path = os.path.join('.', 'output')  # Directory for output

# Ensure output directory exists
os.makedirs(output_path, exist_ok=True)

# Define input file (update this to your file name)
input_file = 'Dataset/test/four_cousins096.jpg'  # Change this to your input file name

# Set paths
input_path = os.path.join(input_path, input_file)
output_file = '{}_out.mp4'.format(os.path.splitext(input_file)[0]) if input_file.lower().endswith(('.mp4', '.avi', '.mov')) else '{}_out.jpg'.format(os.path.splitext(input_file)[0])
output_path = os.path.join(output_path, output_file)

# Load YOLO model
model_path = 'runs/detect/train7/weights/best.pt'
model = YOLO(model_path)
threshold = 0.5

def process_image(image):
    results = model(image)[0]
    four_cousins_count = 0
    pure_heaven_count = 0
    veleta_count = 0

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result
        if score > threshold:
            # Draw the bounding box
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

            # Prepare the label with confidence level in percentage
            label = f'{score*100:.2f}%'

            # Get text size
            (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)

            # Draw a filled rectangle for the label background
            cv2.rectangle(image, (int(x1), int(y1)), (int(x1 + text_width), int(y1 + text_height + 5)), (0, 255, 0), -1)

            # Draw the label inside the box at the top
            cv2.putText(image, label, (int(x1), int(y1 + text_height + 2)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)

            # Count the bottles based on class_id
            if class_id == 0:  # Assuming 0 is the class index for "Four Cousins"
                four_cousins_count += 1
            elif class_id == 1:  # Assuming 1 is the class index for "Pure Heaven"
                pure_heaven_count += 1
            elif class_id == 2:  # Assuming 2 is the class index for "Veletra"
                veleta_count += 1

    return image, four_cousins_count, pure_heaven_count, veleta_count

try:
    if input_file.lower().endswith(('.mp4', '.avi', '.mov')):
        # Video file processing
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            raise IOError("Error: Unable to open video file.")

        ret, frame = cap.read()
        if ret:
            H, W, _ = frame.shape
            out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

            while ret:
                processed_frame, four_cousins_count, pure_heaven_count, veleta_count = process_image(frame)
                out.write(processed_frame)
                ret, frame = cap.read()

            cap.release()
            out.release()
        else:
            raise IOError("Error: Unable to read video file.")
    else:
        # Image file processing
        image = cv2.imread(input_path)
        if image is not None:
            processed_image, four_cousins_count, pure_heaven_count, veleta_count = process_image(image)
            cv2.imwrite(output_path, processed_image)
        else:
            raise IOError("Error: Unable to read image file.")

    print(f"Processing complete. Output saved to {output_path}")
    print(f"Four Cousins count: {four_cousins_count}")
    print(f"Pure Heaven count: {pure_heaven_count}")
    print(f"Veletra count: {veleta_count}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    cv2.destroyAllWindows()